### 単純な多数決分類器を実装する

In [1]:
import numpy as np
np.argmax(np.bincount([0,0,1], weights=[0.2, 0.2, 0.6]))

1

In [2]:
ex = np.array([[0.9, 0.1],
                         [0.8, 0.2],
                         [0.4, 0.6]])

In [3]:
p = np.average(ex, axis=0, weights=[0.2, 0.2, 0.6])
p

array([0.58, 0.42])

In [4]:
np.argmax(p)

0

### MajorityVoteClassifier

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import labelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator

class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    """多数決アンサンブル分類器
    パラメータ
    ------------
    classifiers : array_like, shape = [n_classfiers]
        アンサンブルの様々な分類器
    
    vote : str, {'classlabel', 'probability'}(default, 'classlabel')
        'classlabel'の場合、クラスラベルの予測はクラスラベルのargmaxに基づく
        'probability'の場合、クラスラベルの予測はクラスの所属確率の
        argmaxに基づく（分類器が調節済みであることが推奨される）
        
    """